In [1]:
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows"

import sys
import os
import pandas as pd
from datetime import datetime

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule_weather import generate_activity_list, Scheduler

from plot_weather import plot_weather

In [2]:
#read in activity data from excel
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Activities", skiprows=1)
constraints_df = pd.read_excel(path, sheet_name="Constraints", skiprows=1)

activities = generate_activity_list(act_df, constraints_df)

c:\Users\webbb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
for act in activities:
    print(act.description)
    print(act.duration)
    print(act.constraints)
    print("\n")


Mobilisation of Leask marine spread
5.0
{'daylight_required': False, 'tide_window_required': False, 'max_wind_speed': 12.0, 'max_wave_height': 0.5, 'max_tidal_current': 2.3}


CTV transfer of Personnel  to JUB
2.0
{'daylight_required': True, 'tide_window_required': False, 'max_wind_speed': 10.0, 'max_wave_height': 1.2, 'max_tidal_current': 2.0}


Punch out of pilot
3.0
{'daylight_required': False, 'tide_window_required': False}


Bathymetric survey of pilot to confirm bore location
3.0
{'daylight_required': True, 'tide_window_required': False, 'max_wind_speed': 10.0, 'max_wave_height': 0.7, 'max_tidal_current': 2.0}


Pull back of pilot
6.0
{'daylight_required': False, 'tide_window_required': False}


Unmooring of pipe from Harwich mooring and hook up to tug
1.0
{'daylight_required': False, 'tide_window_required': 'slackhw', 'max_wind_speed': 12.0, 'max_wave_height': 0.5, 'max_tidal_current': 2.3}


Postioning of JUB on site (including transition from Jacked to Floating) for pipe pull


In [ ]:

# 1. Set your project start date
project_start = datetime(2025, 12, 1, 3, 0)  # Example: 1st Dec 2025, 07:00

# 2. Create the Scheduler
scheduler = Scheduler(
    activities=activities,
    weather_data=None,           # Add your weather DataFrame if available
    daylight_csv="sizewell_daylight_windows_december_2025.csv",         # Add daylight windows if available
    tide_csv="lowestoft_slack_water_windows_december_2025.csv",             # Add tide windows if available
    start_datetime=project_start
)

# 3. Run the schedule
scheduled_activities = scheduler.schedule(run_critical_path=True)

# 4. Export to DataFrame for review or export
schedule_df = scheduler.to_dataframe()

# Calculate the total duration by subtracting the earliest start from the latest end
total_duration = schedule_df['End'].max() - schedule_df['Start'].min()

# Print the total duration
print(f"Total duration of the schedule: {total_duration}")

schedule_df.head()


Total duration of the schedule: 5 days 04:17:30


,ID,Description,Duration (hours),Start,End,Group,Predecessor IDs,Constraints,Critical,Tide Window Mismatch
0,PM01,Mobilisation of Leask marine spread,5.0,2025-12-01 07:00:00,2025-12-01 12:00:00,Pipe Management,[],"{'daylight_required': False, 'tide_window_requ...",False,False
1,CTV01,CTV transfer of Personnel to JUB,2.0,2025-12-01 07:40:44,2025-12-01 09:40:44,Crew Transfer Vessel,[],"{'daylight_required': True, 'tide_window_requi...",False,False
2,HDD01,Punch out of pilot,3.0,2025-12-01 12:00:00,2025-12-01 15:00:00,Horizontal Directional Drilling,"[PM01, CTV01]","{'daylight_required': False, 'tide_window_requ...",False,False
3,CTV02,Bathymetric survey of pilot to confirm bore lo...,3.0,2025-12-02 07:42:08,2025-12-02 10:42:08,Crew Transfer Vessel,[HDD01],"{'daylight_required': True, 'tide_window_requi...",False,False
4,HDD02,Pull back of pilot,6.0,2025-12-01 15:00:00,2025-12-01 21:00:00,Horizontal Directional Drilling,[HDD01],"{'daylight_required': False, 'tide_window_requ...",False,False


In [5]:
# Load the tidal data
file_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows\scripts\Tide Data_Lowestoft_10min.CSV"
tide_df = pd.read_csv(file_path, skiprows=3, names=["DateTime", "Height"])

# Convert the 'DateTime' column to datetime format
tide_df['DateTime'] = pd.to_datetime(tide_df['DateTime'], errors='coerce')

# Filter the data to include only rows where the month is December
december_tide_df = tide_df[tide_df['DateTime'].dt.month == 12]


In [6]:
tide_window_df = pd.read_csv("lowestoft_slack_water_windows_december_2025.csv")

In [7]:
plot_weather(schedule_df, scheduler.daylight_windows, december_tide_df, tide_window_df)